In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

In [2]:
CLIENT_ID = 'XHHD3PX12PTV0QUO103UXRMTHOBDKZ3G351JAMIWPK4BICCS' # your Foursquare ID
CLIENT_SECRET = '0E5SFDT3HB1VR1X0DU1NV4B3D2SC1OV0KYM30PA1MEPVJOKG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XHHD3PX12PTV0QUO103UXRMTHOBDKZ3G351JAMIWPK4BICCS
CLIENT_SECRET:0E5SFDT3HB1VR1X0DU1NV4B3D2SC1OV0KYM30PA1MEPVJOKG


In [3]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [4]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&sortByDistance=True'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XHHD3PX12PTV0QUO103UXRMTHOBDKZ3G351JAMIWPK4BICCS&client_secret=0E5SFDT3HB1VR1X0DU1NV4B3D2SC1OV0KYM30PA1MEPVJOKG&v=20180605&ll=40.7900869,-73.9598295&radius=1000&limit=100&sortByDistance=True'

In [63]:
results = requests.get(url).json()

# define the dataframe columns
column_names = ['Name', 'Latitude', 'Longitude', 'Distance', 'CategoryName'] 

# instantiate the dataframe
outcome = pd.DataFrame(columns=column_names)

venues = results['response']['groups'][0]['items']
for venue in venues:
    name = venue['venue']['name'] #name
    lat = venue['venue']['location']['lat'] #latitude
    long = venue['venue']['location']['lng'] #lng
    distance = venue['venue']['location']['distance'] # distance
    categoryname = venue['venue']['categories'][0]['name'] #category name
    outcome = outcome.append({'Name': name,
                              'Latitude': lat,
                              'Longitude': long,
                              'Distance': distance,
                              'CategoryName': categoryname}, ignore_index=True)
outcome

,Name,Latitude,Longitude,Distance,CategoryName
0,Central Park Tennis Center,40.789313,-73.961862,191,Tennis Court
1,North Meadow,40.792027,-73.959853,215,Park
2,Central Park - North Meadow Fields 1-4,40.792128,-73.961362,261,Baseball Field
3,East Meadow,40.790160,-73.955498,365,Park
4,Central Park - 96th Street Playground,40.787845,-73.956369,383,Playground
5,Rudin Family Playground,40.791822,-73.964147,411,Playground
6,Central Park - 99th & 5th Ave,40.788791,-73.955232,413,Park
7,Central Park - Woodman's Gate,40.787786,-73.955924,417,Park
8,Central Park - Gate Of All Saints,40.791591,-73.964795,450,Park
9,Tarr Family Playground,40.793706,-73.962432,458,Playground


In [66]:
#create map of manhattan with markers for venues
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, categoryname in zip(outcome['Latitude'], outcome['Longitude'], outcome['CategoryName']):
    label = '{}'.format(categoryname)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [64]:
categorynamedf = outcome[['CategoryName']]
categorynamedf['Number of Occurences'] = 1
categorynamedf = categorynamedf.groupby('CategoryName').count()
categorynamedf['Percentage of Occurences'] = categorynamedf['Number of Occurences']/(categorynamedf['Number of Occurences'].sum())
categorynamedf = categorynamedf.sort_values(by='Percentage of Occurences', ascending=False)
categorynamedf

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Number of Occurences,Percentage of Occurences
CategoryName,,
Park,9,0.09
Playground,6,0.06
Scenic Lookout,5,0.05
Café,5,0.05
Plaza,4,0.04
Wine Shop,4,0.04
Grocery Store,4,0.04
Baseball Field,3,0.03
Art Museum,3,0.03
